In [ ]:
class Line(object):
    def __init__(self):
        # was the line detected in the last frame
        self.detected = False
        # x values of the last n fits
        self.recent_xfitted = []
        # average x values of the fitted line, over the last n iterations
        self.bestx = None
        # polynomial co-efficients averaged over the last n iterations
        self.best_fit = None
        # polynomial co-efficients for the most recent fit
        self.current_fit = np.array([False])
        # radius of curvature of the line 
        self.radius_of_curvature = None
        # distance in meters of vehicle center from the lane
        self.line_base_pos = None
        # difference in fit co-efficients between the current and last fit
        self.diffs = np.array([0, 0, 0], dtype = "float")
        # x values for detected line pixels
        self.allx = None
        # y values for detected line pixels
        self.ally = None
    
    # Sliding Window Method
    # This involves defining a window (rectangle) for each lane. Which starts at the bottom and slides to the top of the lane. 
    # After each slide. We recalculate the center of the window, using the average x co-ordinate of the non-zero pixels
    # within the window.
class Find_Lanes():
    def __init__(self):
        self.left_lane = Line()
        self.right_lane = Line()
    def find_lane_lines_new_fit(self, binary_warped, plot = True):
        # First define an output image, on which we can visualize the windows and lanes
        output_image = np.dstack((binary_warped, binary_warped, binary_warped)) # 3 channel image
        # Next we have to define the midpoint for each lane.
        midpoint = np.int(binary_warped.shape[1] / 2)
        leftx_mid = np.argmax(histogram[:midpoint])
        rightx_mid = np.argmax(histogram[midpoint:]) + midpoint
        print(leftx_mid), print(rightx_mid)
        # Let us define what a window is
        nWindows = 10 # This can be adjusted
        window_height = np.int(binary_warped.shape[0] / nWindows)
        margin = 100 # this can be adjusted

        # Let us get all x and y co-ordinates of non zero pixels
        non_zero = binary_warped.nonzero()
        non_zeroy = np.array(non_zero[0])
        non_zerox = np.array(non_zero[1])
    
        # Current positions of windows, these will be updated once we find a windows with a min number of pixels
        leftx_current = leftx_mid
        rightx_current = rightx_mid
        min_pixel = 50 # this can be adjusted

        # Empty lists to store the left and right lanes indices
        left_lane_inds = []
        right_lane_inds = []

        for window in range(nWindows):
            # Define window boundaries
            window_ylow = binary_warped.shape[0] - (window + 1)*(window_height)
            window_yhigh = binary_warped.shape[0] - (window)*(window_height)
            window_xleft_low = leftx_current - margin    
            window_xleft_high = leftx_current + margin
            window_xright_low = rightx_current - margin
            window_xright_high = rightx_current + margin
            # Draw the windows on the output image
            cv2.rectangle(output_image,(window_xleft_low,window_ylow),(window_xleft_high,window_yhigh),
            (0,255,0), 2) 
            cv2.rectangle(output_image,(window_xright_low,window_ylow),(window_xright_high,window_yhigh),
            (0,255,0), 2) 
            # Find the left and right non zero indices which are in each of the windows
            good_left_inds = ((non_zeroy >= window_ylow) & (non_zeroy < window_yhigh) & 
                             (non_zerox >= window_xleft_low) &  (non_zerox < window_xleft_high)).nonzero()[0]
    
            good_right_inds = ((non_zeroy >= window_ylow) & (non_zeroy < window_yhigh) &
                              (non_zerox >= window_xright_low) & (non_zerox < window_xright_high)).nonzero()[0]
            #print(good_right_inds)
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If the window has enough pixels, update the x position of the lane
            if len(good_left_inds) > min_pixel:        
                leftx_current = np.int(np.mean(non_zerox[good_left_inds]))
            if len(good_right_inds) > min_pixel:
                rightx_current = np.int(np.mean(non_zerox[good_right_inds]))
        
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        # Extract the pixel positions for each lane and store the in the line objects
        leftx = non_zerox[left_lane_inds]
        lefty = non_zeroy[left_lane_inds]
        self.left_lane.allx = leftx
        self.left_lane.ally = lefty
        
        rightx = non_zerox[right_lane_inds]
        righty = non_zeroy[right_lane_inds]
        self.right_lane.allx = rightx
        self.right_lane.ally = righty
        # We are fitting a polynomial for each lane. We will use these co-efficients to predict an x co-ordinate
        # for a given y co-ordinate. Hence the order of inputs to np.polyfit, first y co-ordinates and then x.
        
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        self.left_lane.current_fit = left_fit
        sefl.right_lane.current_fit = right_fit
        
        # Now let us color all the non zero pixels which belong to left and right lane lines in the output image
        output_image[non_zeroy[left_lane_inds], non_zerox[left_lane_inds]] = (255, 0, 0)
        output_image[non_zeroy[right_lane_inds], non_zerox[right_lane_inds]] = (0, 0, 255)
        ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])    
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] # x = ay^2 + by + c (parabola fit)
        
        # Now we have the got the x values from the fit, lets append to the left and right lanes
        self.left_lane.recent_xfitted.append(left_fitx)        
        self.right_lane.recent_xfitted.append(right_fitx)
        
        if len(self.left_lane.recent_xfitted) > 1:
            self.left_lane.bestx = np.mean(self.left_lane.recent_xfitted, axis = 0)
        if len(self.right_lane.recent_xfitted) > 1:
            self.right_lane.bestx = np.mean(self.right_lane.recent_xfitted, axis = 0)
        
        # For future radius of curvature calculations, lets find left_fitcr and right_fitcr
        ym_per_pix = 30/720
        xm_per_pix = 3.7/(rightx_mid - leftx_mid)
        left_fitcr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
        right_fitcr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
        radius_of_curvature(left_fitcr, right_fitcr)
        if plot == True:
            plt.imshow(output_image)
            plt.plot(left_fitx, ploty, "y")
            plt.plot(right_fitx, ploty, "y")
            plt.xlim([0, 1280])
            plt.ylim([720, 0])
        return left_fit, right_fit, leftx, rightx, left_fitcr, right_fitcr
left_fit, right_fit = find_lane_lines_new_fit(binary_warped)
    
    

        
        
        